<a href="https://colab.research.google.com/github/denzilsaldanha/monument/blob/master/Transfer_Learning_Mobile_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
from PIL import Image
count = 0
for folder in os.listdir('drive/My Drive/train_sample_images/'):
    for file in os.listdir('drive/My Drive/train_sample_images/'+str(folder)):
        if file.endswith('.jpg') or file.endswith('.mpo'):
            try:
                img = Image.open('drive/My Drive/train_sample_images/'+str(folder)+'/'+str(file)) # open the image file
                img.verify() # verify that it is, in fact an image
            except (IOError, SyntaxError) as e:
                count +=1
                os.remove('drive/My Drive/train_sample_images/'+str(folder)+'/'+str(file))
                print('Bad file: ', str(file))
                
                
print("CHECKED ALL FILES")

In [0]:

import pandas as pd
import numpy as np
import glob
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam


# In[2]:


base_model=MobileNetV2(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(150,activation='softmax')(x) #final layer with softmax activation


# In[3]:


model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture
layers = 0


# In[4]:

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    layers += 1
print(layers)


# In[5]:


train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2) #included in our dependencies

# train_generator=train_datagen.flow_from_directory('./train/', # this is where you specify the path to the main data folder
#                                                  target_size=(224,224),
#                                                  color_mode='rgb',
#                                                  batch_size=32,
#                                                  class_mode='categorical',
#                                                  shuffle=True)


training_set = train_datagen.flow_from_directory('drive/My Drive/train_sample_images',
                                                 target_size = (224, 224),
                                                 batch_size = 128,
                                                 class_mode = 'categorical',
                                                subset ='training')
validation_set = train_datagen.flow_from_directory(
    'drive/My Drive/train_sample_images', # same directory as training data
    target_size=(224,224),
    batch_size=128,
    class_mode='categorical',
    subset='validation')

# In[33]:
epochs_run = 0

# if glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-mobile_net-[0-9]{1,2}[.-].*'):
#   #max(glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-mobile-net-[0-9][0-9]*'))
#   weight_to_be_loaded = max(glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-mobile-net-[0-9]{1,2}[.-].*'))
#   weight_to_be_loaded = model.load_weights('/drive/My Drive/GOOGLE_MONUMENT/weights-improvement-mobile_net-01-0.36.hdf5')
#   model.load_weights(weight_to_be_loaded)
# #   epochs_run = int(weight_to_be_loaded[62:64])
# if os.path.isfile('/drive/My Drive/GOOGLE_MONUMENT/model_h5_mobile_net.h5'): # if thats the case then there must be a read error somewhere.
#   model.load_weights('/drive/My Drive/GOOGLE_MONUMENT/model_h5_mobile_net.h5')
#   epochs_run = 5
#   # Change this to the file saved by the second epoch and initiallize epochs run to 1+ the number of epochs run. 
#   #Basically it should start from the 3rd epoch if youre loading the 2 epoch weights
# else:
#   print('MODEL NOT FOUND')
  
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy


csv_file = 'drive/My Drive/GOOGLE_MONUMENT/log_for_mobile_net_starting_from_epoch_'+str(epochs_run)+'.csv'
checkpoint="drive/My Drive/GOOGLE_MONUMENT/weights-improvement-mobile_net-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint, monitor='val_acc', verbose=1, mode='max')
csv_logger = CSVLogger(csv_file, append=True, separator=';')
callbacks_list = [csv_logger,checkpoint]

#model.summary()
step_size_train=training_set.n//training_set.batch_size
val_steps = validation_set.n//validation_set.batch_size
model.fit_generator(generator=training_set,
                   steps_per_epoch=step_size_train,
                   epochs=5, validation_data = validation_set,validation_steps=val_steps,
                   callbacks =callbacks_list,initial_epoch = epochs_run)


#print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
model.save('drive/My Drive/GOOGLE_MONUMENT/model_mobilenet.hdf5')
model.save_weights("drive/My Drive/GOOGLE_MONUMENT/model_h5_mobile_net.h5")
print("Saved model to disk") 
# serialize model to JSON
model_json = model.to_json()
with open("drive/My Drive/GOOGLE_MONUMENT/model_mobilenet.json", "w") as json_file:
    json_file.write(model_json)




